In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Spark Oturumunu Başlatıyoruz
spark = SparkSession.builder \
    .appName("ALAZ_Test") \
    .master("local[*]") \
    .getOrCreate()

print("🔥 Ubuntu üzerinde ALAZ Sistemi Devrede! Spark Versiyonu:", spark.version)

# Basit bir test verisi
data = [("Muğla", 1), ("İstanbul", 0), ("Antalya", 1)]
columns = ["Sehir", "Yangin_Durumu"]

df = spark.createDataFrame(data, columns)
df.show()

25/12/13 21:10:34 WARN Utils: Your hostname, turan-H110 resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface enp2s0)
25/12/13 21:10:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/13 21:10:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


🔥 Ubuntu üzerinde ALAZ Sistemi Devrede! Spark Versiyonu: 3.5.1


+--------+-------------+
|   Sehir|Yangin_Durumu|
+--------+-------------+
|   Muğla|            1|
|İstanbul|            0|
| Antalya|            1|
+--------+-------------+



In [4]:
from pyspark.sql.functions import col, to_date, lpad, substring

# 1. CSV Dosyasını Oku
df_fire = spark.read.csv("../data/raw/modis_turkey.csv", header=True, inferSchema=True)

# 2. Saati Düzeltme İşlemi (Kritik Hamle)
# NASA saati 135 olarak verebiliyor (01:35 demek). Bunu 4 haneli '0135' stringine çeviriyoruz.
# lpad(col("acq_time"), 4, "0") -> Soluna 0 koyarak 4 hane yap demek.
df_fire = df_fire.withColumn("acq_time_str", lpad(col("acq_time").cast("string"), 4, "0"))

# 3. Sütun Seçimi (Saati de ekledik)
fire_clean = df_fire.filter(col("confidence") >= 50) \
    .select(
        col("latitude"),
        col("longitude"),
        to_date(col("acq_date"), "yyyy-MM-dd").alias("tarih"),
        
        # Tam saat verisi (Örn: "1335")
        col("acq_time_str").alias("saat_dakika_raw"), 
        
        # Bize lazım olan Sadece SAAT kısmı (Örn: 13). Çünkü ERA5 verisi saatlik.
        # İlk 2 karakteri alıp integer yapıyoruz.
        substring(col("acq_time_str"), 1, 2).cast("int").alias("saat"),
        
        col("frp").alias("yangin_siddeti"),
        col("confidence")
    )

print(f"Güncellenmiş Temiz Veri Sayısı: {fire_clean.count()}")

# 4. Tabloya bir bakalım, saat gelmiş mi?
fire_clean.show(10)

# 5. Üzerine Yazarak Kaydet
fire_clean.write.mode("overwrite").parquet("../data/processed/fire_data.parquet")

print("✅ Saat verisiyle birlikte güncelleme tamam!")

Güncellenmiş Temiz Veri Sayısı: 202014
+--------+---------+----------+---------------+----+--------------+----------+
|latitude|longitude|     tarih|saat_dakika_raw|saat|yangin_siddeti|confidence|
+--------+---------+----------+---------------+----+--------------+----------+
| 36.8527|  35.3594|2000-11-01|           0815|   8|          13.2|        60|
| 40.5421|  35.1909|2000-11-02|           0857|   8|           7.3|        69|
| 40.5441|  35.1786|2000-11-02|           0857|   8|           5.8|        66|
| 39.4256|  34.1707|2000-11-02|           0858|   8|           8.5|        69|
| 39.4577|  32.7936|2000-11-02|           0858|   8|           5.9|        65|
| 39.9923|  33.6773|2000-11-02|           0858|   8|           8.9|        69|
| 39.8424|  32.4918|2000-11-02|           0858|   8|           5.8|        64|
|   39.57|  34.0749|2000-11-02|           0858|   8|           4.8|        59|
| 39.1981|  32.4182|2000-11-02|           0858|   8|           6.1|        64|
| 39.1714|  3

[Stage 18:==============>                                           (1 + 3) / 4]

✅ Saat verisiyle birlikte güncelleme tamam!


In [6]:
import random
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, DateType, IntegerType
from datetime import date, timedelta

# 1. Kaç tane "Yangın Yok (0)" verisi üretelim?
# Yangın sayısı kadar (202.000) üretelim ki veri dengeli (balanced) olsun.
num_samples = 205000 

print(f"Hedef: {num_samples} adet negatif (yangın olmayan) veri üretiliyor...")

# 2. Rastgele Veri Üretme Fonksiyonları
def generate_random_data():
    data = []
    start_date = date(2000, 1, 1)
    end_date = date(2023, 12, 31)
    days_range = (end_date - start_date).days

    for _ in range(num_samples):
        # Türkiye'nin Kabaca Koordinatları: Enlem (36-42), Boylam (26-45)
        # uniform(a, b) -> a ile b arasında rastgele ondalıklı sayı üretir
        lat = random.uniform(36.0, 42.0)
        lon = random.uniform(26.0, 45.0)
        
        # Rastgele Tarih
        random_days = random.randint(0, days_range)
        random_date = start_date + timedelta(days=random_days)
        
        # Rastgele Saat (00 ile 23 arası)
        random_hour = random.randint(0, 23)
        
        # "yangin_siddeti" (frp) 0 olacak, "confidence" 0 olacak.
        # Format: (lat, lon, tarih, raw_time, saat, frp, confidence, LABEL)
        # Not: Label (0) sütununu sonra ekleyeceğiz, şimdilik şemayı yangın verisiyle aynı tutuyoruz.
        data.append((lat, lon, random_date, "0000", random_hour, 0.0, 0))
    
    return data

# 3. Veriyi Oluştur
random_data = generate_random_data()

# 4. Şemayı Tanımla (Yangın verisiyle birebir aynı olmalı)
schema = StructType([
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("tarih", DateType(), True),
    StructField("saat_dakika_raw", StringType(), True),
    StructField("saat", IntegerType(), True),
    StructField("yangin_siddeti", DoubleType(), True),
    StructField("confidence", IntegerType(), True)
])

# 5. DataFrame'e Çevir
df_non_fire = spark.createDataFrame(random_data, schema)

# 6. Etiketleme (Labeling)
# Yangın verisine "label = 1", Buna "label = 0" ekleyeceğiz.
from pyspark.sql.functions import lit

# Mevcut yangın verisini (diskten) tekrar okuyalım
df_fire = spark.read.parquet("../data/processed/fire_data.parquet")
df_fire_labeled = df_fire.withColumn("label", lit(1))

# Negatif veriye label ekleyelim
df_non_fire_labeled = df_non_fire.withColumn("label", lit(0))

print("🔥 Pozitif Veri (Yangın):")
df_fire_labeled.show(2)

print("🌲 Negatif Veri (Normal):")
df_non_fire_labeled.show(2)

# 7. İkisini Birleştir (Merge)
df_final = df_fire_labeled.union(df_non_fire_labeled)

print(f"Toplam Veri Seti Boyutu: {df_final.count()}")

# 8. Hepsini Tek Parquet Olarak Kaydet
df_final.write.mode("overwrite").parquet("../data/processed/training_dataset_v1.parquet")

print("✅ Hazırlık Tamam! 'training_dataset_v1.parquet' içinde 1 ve 0 verileri birleşti.")

Hedef: 205000 adet negatif (yangın olmayan) veri üretiliyor...
🔥 Pozitif Veri (Yangın):
+--------+---------+----------+---------------+----+--------------+----------+-----+
|latitude|longitude|     tarih|saat_dakika_raw|saat|yangin_siddeti|confidence|label|
+--------+---------+----------+---------------+----+--------------+----------+-----+
| 36.8527|  35.3594|2000-11-01|           0815|   8|          13.2|        60|    1|
| 40.5421|  35.1909|2000-11-02|           0857|   8|           7.3|        69|    1|
+--------+---------+----------+---------------+----+--------------+----------+-----+
only showing top 2 rows

🌲 Negatif Veri (Normal):
+------------------+------------------+----------+---------------+----+--------------+----------+-----+
|          latitude|         longitude|     tarih|saat_dakika_raw|saat|yangin_siddeti|confidence|label|
+------------------+------------------+----------+---------------+----+--------------+----------+-----+
|39.421053173841145| 32.42740175471714|2

25/12/13 21:33:24 WARN TaskSetManager: Stage 21 contains a task of very large size (1959 KiB). The maximum recommended task size is 1000 KiB.
25/12/13 21:33:24 WARN TaskSetManager: Stage 22 contains a task of very large size (1959 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

Toplam Veri Seti Boyutu: 407014


25/12/13 21:33:25 WARN TaskSetManager: Stage 25 contains a task of very large size (1959 KiB). The maximum recommended task size is 1000 KiB.
[Stage 25:=============================>                            (4 + 4) / 8]

✅ Hazırlık Tamam! 'training_dataset_v1.parquet' içinde 1 ve 0 verileri birleşti.


In [7]:
!pip install earthengine-api geemap

  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached googleapis_common_protos-1.72.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached contourpy-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (6.3 kB)
  Using cached pillow-12.0.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.8 kB)
Using cached pyparsing-3.2.5-py3-none-any.whl (113 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 54.3 MB/s  0:00:00 65.7 MB/s eta 0:00:01
Using cached googleapis_common_protos-1.72.0-py3-none-any.whl (297 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.4/863.4 kB 58.6 MB/s  0:00:00
   ━━━━━━━━━━━

In [8]:
import ee

# Bu komut bir link verecek ve senden bir şifre (token) isteyecek.
try:
    ee.Initialize()
    print("Google Earth Engine zaten aktif! 🌍")
except Exception as e:
    print("Yetki veriliyor...")
    ee.Authenticate()
    ee.Initialize()
    print("Başarıyla bağlandı! 🚀")

/home/turan/anaconda3/envs/alaz_env/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


Yetki veriliyor...


Enter verification code:  4/1ATX87lNeamCMnNBaT6GdVn-OJVcn-Zi2bXDn4NuaegSnRM7SlCegXpVhw0g



Successfully saved authorization token.
Başarıyla bağlandı! 🚀


In [9]:
import ee

# Eğer yukarıda initialize ettiysen tekrar etmene gerek yok ama garanti olsun
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

# Manavgat civarından bir nokta (Enlem, Boylam)
point = ee.Geometry.Point([31.45, 36.85]) 

# 1. Yükselti Verisi (SRTM - NASA)
dem = ee.Image('USGS/SRTMGL1_003')
elevation = dem.sample(point, 30).first().get('elevation').getInfo()

# 2. Sıcaklık Verisi (ERA5 - ECMWF) - Yangın zamanı (28 Temmuz 2021)
dataset = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
            .filterDate('2021-07-28', '2021-07-29') \
            .first() 

temp_kelvin = dataset.sample(point, 11000).first().get('temperature_2m').getInfo()
temp_celsius = temp_kelvin - 273.15

print("✅ SİSTEM AKTİF!")
print(f"📍 Koordinat: Manavgat")
print(f"⛰️  Yükselti: {elevation} metre")
print(f"🌡️  Sıcaklık: {temp_celsius:.2f} °C")

✅ SİSTEM AKTİF!
📍 Koordinat: Manavgat
⛰️  Yükselti: 33 metre
🌡️  Sıcaklık: 28.61 °C


In [11]:
import ee
import pandas as pd
import numpy as np
from pyspark.sql.functions import col

# Spark'tan veriyi okuyalım (Zaten hazırdı)
df_spark = spark.read.parquet("../data/processed/training_dataset_v1.parquet")

# Test için ilk 1000 satırı alalım (Hepsini birden gönderirsek GEE bazen timeout verir, parça parça yapacağız)
# Şimdilik sistem çalışıyor mu diye 100 satırla deniyoruz.
sample_data = df_spark.limit(100).toPandas()

print(f"📡 {len(sample_data)} adet nokta GEE'ye gönderiliyor...")

def get_terrain_features(df):
    # 1. Pandas DataFrame'i GEE FeatureCollection'a çevir
    features = []
    for index, row in df.iterrows():
        g = ee.Geometry.Point([row['longitude'], row['latitude']])
        f = ee.Feature(g, {'id': str(index)}) # ID'yi saklayalım ki karışmasın
        features.append(f)
    
    fc = ee.FeatureCollection(features)

    # 2. SRTM Verisini Çağır (Yükselti)
    srtm = ee.Image("USGS/SRTMGL1_003")
    
    # 3. Türetilmiş Verileri Hesapla (Eğim ve Bakı)
    slope = ee.Terrain.slope(srtm)
    aspect = ee.Terrain.aspect(srtm)
    
    # 4. Hepsini Tek Görüntüde Birleştir
    terrain_image = ee.Image.cat([srtm, slope, aspect]).rename(['yukselti', 'egim', 'baki'])
    
    # 5. Noktalar üzerindeki değerleri çek (reduceRegions)
    # scale=30 -> 30 metre çözünürlükte çek demek
    results = terrain_image.reduceRegions(collection=fc, reducer=ee.Reducer.first(), scale=30)
    
    # 6. Sonucu Geri Al ve Pandas'a Çevir
    # getInfo() veriyi Google'dan senin bilgisayara indirir.
    info = results.getInfo()
    
    # Gelen JSON'ı listeye çevir
    extracted_data = []
    for f in info['features']:
        props = f['properties']
        extracted_data.append(props)
        
    return pd.DataFrame(extracted_data)

# Fonksiyonu Çalıştır
df_terrain = get_terrain_features(sample_data)

# Sonuçları Birleştir
# Orijinal veriyle GEE'den gelen veriyi yan yana koyuyoruz
df_final_sample = pd.concat([sample_data.reset_index(drop=True), df_terrain.reset_index(drop=True)], axis=1)

print("✅ Topografik veriler çekildi!")
df_final_sample[['latitude', 'longitude', 'yukselti', 'egim', 'baki']].head(10)

📡 100 adet nokta GEE'ye gönderiliyor...
✅ Topografik veriler çekildi!


,latitude,longitude,yukselti,egim,baki
0,36.674510,28.089241,63,13.731303,277.613803
1,37.362840,39.682859,737,1.854334,0.000000
2,39.746110,32.855580,1147,1.854334,180.000000
3,38.864478,35.555199,1087,0.000000,0.000000
4,39.998942,40.995847,1865,2.214035,33.132204
5,40.253884,32.167876,795,12.989641,55.847429
6,37.582413,33.067022,998,2.192266,32.249219
7,38.213384,32.471280,1130,2.359555,270.000000
8,40.302604,31.296692,754,0.000000,0.000000
9,40.422306,33.106415,1471,10.362758,152.281392


In [14]:
import math

print("⛈️ Meteorolojik veriler (ERA5) - Tamir Edilmiş Modda Çekiliyor...")

def get_weather_features(df):
    features = []
    for index, row in df.iterrows():
        # Tarih ve Saati birleştir
        dt_str = f"{row['tarih']} {row['saat']}:00:00"
        millis = pd.Timestamp(dt_str).value // 10**6
        
        g = ee.Geometry.Point([row['longitude'], row['latitude']])
        f = ee.Feature(g, {'system:time_start': millis, 'id': str(index)})
        features.append(f)

    fc = ee.FeatureCollection(features)

    # --- GÜNCELLENMİŞ FONKSİYON ---
    def extract_weather(feature):
        date_millis = feature.get('system:time_start')
        date = ee.Date(date_millis)
        
        # Görüntü koleksiyonunu çağır
        img_col = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
                .filterDate(date, date.advance(1, 'hour'))
        
        # Band İsimleri Listesi
        band_names = ['temperature_2m', 'u_component_of_wind_10m', 'v_component_of_wind_10m', 'dewpoint_temperature_2m', 'total_precipitation_hourly']
        
        # DÜZELTİLEN KISIM BURASI:
        # Eğer görüntü yoksa, 5 tane 0 içeren bir liste veriyoruz ki 5 band oluşsun.
        fallback_img = ee.Image.constant([0, 0, 0, 0, 0]).rename(band_names)
        
        img = ee.Image(ee.Algorithms.If(
            img_col.size().gt(0),
            img_col.first(),
            fallback_img # Boşsa bunu kullan
        ))
        
        # Seç ve Maskeleme Kaldır (-999 yap)
        img = img.select(band_names).unmask(-999)
        
        data = img.reduceRegion(reducer=ee.Reducer.first(), geometry=feature.geometry(), scale=11000)
        
        # --- FİZİK HESAPLAMALARI (Null Kontrollü) ---
        
        # Sıcaklık 
        t_raw = ee.Number(data.get('temperature_2m'))
        # Eğer veri yoksa (-999 veya 0) sonuç -999 kalsın
        temp_c = ee.Algorithms.If(t_raw.eq(-999).Or(t_raw.eq(0)), -999, t_raw.subtract(273.15))
        
        # Rüzgar
        u = ee.Number(data.get('u_component_of_wind_10m'))
        v = ee.Number(data.get('v_component_of_wind_10m'))
        # Eğer u veya v -999 ise sonuç -999
        wind_speed = ee.Algorithms.If(u.eq(-999).Or(u.eq(0)), -999, (u.pow(2).add(v.pow(2))).sqrt())
        
        # Nem
        dew_raw = ee.Number(data.get('dewpoint_temperature_2m'))
        
        rh = ee.Algorithms.If(
            t_raw.eq(-999).Or(dew_raw.eq(-999)).Or(t_raw.eq(0)), 
            -999,
            ee.Number(100).multiply(
                ((ee.Number(17.625).multiply(dew_raw.subtract(273.15))).divide(ee.Number(243.04).add(dew_raw.subtract(273.15)))).exp() \
                .divide(
                    ((ee.Number(17.625).multiply(t_raw.subtract(273.15))).divide(ee.Number(243.04).add(t_raw.subtract(273.15)))).exp()
                )
            )
        )
        
        # Yağış
        p_raw = ee.Number(data.get('total_precipitation_hourly'))
        # Yağışta 0 gerçek bir değer olabilir (yağmur yok), o yüzden sadece -999 kontrolü yapıyoruz.
        precip = ee.Algorithms.If(p_raw.eq(-999), -999, p_raw.multiply(1000))
        
        return feature.set({
            'sicaklik': temp_c,
            'ruzgar_hizi': wind_speed,
            'bagil_nem': rh,
            'yagis': precip
        })

    # Map işlemini yap
    processed_fc = fc.map(extract_weather, True)
    
    info = processed_fc.getInfo()
    
    extracted_data = []
    for f in info['features']:
        props = f['properties']
        extracted_data.append(props)
        
    return pd.DataFrame(extracted_data)

# Testi Tekrar Başlat
df_weather = get_weather_features(df_final_sample)

# Birleştirme
df_final_v2 = pd.concat([df_final_sample, df_weather[['sicaklik', 'ruzgar_hizi', 'bagil_nem', 'yagis']]], axis=1)

print("✅ Meteorolojik veriler başarıyla işlendi!")
df_final_v2.head(10)

⛈️ Meteorolojik veriler (ERA5) - Tamir Edilmiş Modda Çekiliyor...
✅ Meteorolojik veriler başarıyla işlendi!


,latitude,longitude,tarih,saat_dakika_raw,saat,yangin_siddeti,confidence,label,baki,egim,id,yukselti,sicaklik,ruzgar_hizi,bagil_nem,yagis
0,36.674510,28.089241,2017-08-30,0000,20,0.0,0,0,277.613803,13.731303,0,63,-999.000000,-999.000000,-999.000000,-999.000000
1,37.362840,39.682859,2016-12-21,0000,16,0.0,0,0,0.000000,1.854334,1,737,2.074533,2.612000,64.792462,0.027972
2,39.746110,32.855580,2009-06-01,0000,3,0.0,0,0,180.000000,1.854334,2,1147,10.644830,1.494043,81.356549,0.000000
3,38.864478,35.555199,2009-01-23,0000,6,0.0,0,0,0.000000,0.000000,3,1087,-2.944785,2.536245,47.445634,0.000000
4,39.998942,40.995847,2000-03-26,0000,20,0.0,0,0,33.132204,2.214035,4,1865,-2.838187,0.968401,64.041746,0.000000
5,40.253884,32.167876,2018-09-03,0000,16,0.0,0,0,55.847429,12.989641,5,795,27.346964,1.189320,26.501373,0.000000
6,37.582413,33.067022,2001-03-23,0000,13,0.0,0,0,32.249219,2.192266,6,998,12.448145,4.200345,59.681215,1.022279
7,38.213384,32.471280,2013-09-05,0000,19,0.0,0,0,270.000000,2.359555,7,1130,14.290872,2.357577,37.053009,0.000000
8,40.302604,31.296692,2023-02-07,0000,15,0.0,0,0,0.000000,0.000000,8,754,-6.058112,2.922375,71.133840,0.024967
9,40.422306,33.106415,2007-10-08,0000,2,0.0,0,0,152.281392,10.362758,9,1471,9.424173,1.618959,72.224117,0.000000


In [15]:
print("🌿 Bitki Örtüsü (NDVI) verileri çekiliyor...")

def get_ndvi_features(df):
    features = []
    for index, row in df.iterrows():
        dt_str = f"{row['tarih']} {row['saat']}:00:00"
        millis = pd.Timestamp(dt_str).value // 10**6
        g = ee.Geometry.Point([row['longitude'], row['latitude']])
        f = ee.Feature(g, {'system:time_start': millis, 'id': str(index)})
        features.append(f)

    fc = ee.FeatureCollection(features)

    def extract_ndvi(feature):
        date_millis = feature.get('system:time_start')
        date = ee.Date(date_millis)
        
        # MODIS NDVI (16 Günlük periyotlarla üretilir)
        # Yangın tarihinden önceki son görüntüyü almak en mantıklısı.
        # Tarihten geriye 20 gün gidip arıyoruz.
        img_col = ee.ImageCollection("MODIS/006/MOD13Q1") \
                .filterDate(date.advance(-20, 'day'), date) \
                .sort('system:time_start', False) # En yenisi en başta olsun
        
        # Fallback (Görüntü yoksa -999)
        fallback_img = ee.Image.constant([0]).rename(['NDVI'])
        
        img = ee.Image(ee.Algorithms.If(
            img_col.size().gt(0),
            img_col.first(), # En yakın tarihli görüntüyü al
            fallback_img
        ))
        
        # MODIS NDVI verisi 0.0001 ile çarpılmalı (Scale Factor)
        # Ayrıca -999 maskelemesi yapıyoruz.
        raw_ndvi = img.select('NDVI').unmask(-999)
        
        # Değeri çek
        data = raw_ndvi.reduceRegion(reducer=ee.Reducer.first(), geometry=feature.geometry(), scale=250)
        
        ndvi_val = ee.Number(data.get('NDVI'))
        
        # Eğer -999 değilse scale et, yoksa -999 bırak
        final_ndvi = ee.Algorithms.If(ndvi_val.eq(-999), -999, ndvi_val.multiply(0.0001))
        
        return feature.set({'NDVI': final_ndvi})

    processed_fc = fc.map(extract_ndvi, True)
    info = processed_fc.getInfo()
    
    extracted_data = []
    for f in info['features']:
        props = f['properties']
        extracted_data.append(props)
        
    return pd.DataFrame(extracted_data)

# Testi Çalıştır (df_final_sample üzerinde, yani ilk 100 satır)
df_ndvi = get_ndvi_features(df_final_sample)

# Ana Tabloya Ekle
df_final_v3 = pd.concat([df_final_v2, df_ndvi[['NDVI']]], axis=1)

print("✅ NDVI verileri eklendi!")
df_final_v3[['tarih', 'latitude', 'longitude', 'sicaklik', 'NDVI']].head(10)

🌿 Bitki Örtüsü (NDVI) verileri çekiliyor...


/home/turan/anaconda3/envs/alaz_env/lib/python3.10/site-packages/ee/deprecation.py:215: DeprecationWarning: 

Attention required for MODIS/006/MOD13Q1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
This dataset has been superseded by MODIS/061/MOD13Q1

Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13Q1

  warnings.warn(warning, category=DeprecationWarning)


✅ NDVI verileri eklendi!


,tarih,latitude,longitude,sicaklik,NDVI
0,2017-08-30,36.674510,28.089241,-999.000000,0.3260
1,2016-12-21,37.362840,39.682859,2.074533,0.2149
2,2009-06-01,39.746110,32.855580,10.644830,0.2060
3,2009-01-23,38.864478,35.555199,-2.944785,0.2768
4,2000-03-26,39.998942,40.995847,-2.838187,-0.0614
5,2018-09-03,40.253884,32.167876,27.346964,0.2945
6,2001-03-23,37.582413,33.067022,12.448145,0.1691
7,2013-09-05,38.213384,32.471280,14.290872,0.1528
8,2023-02-07,40.302604,31.296692,-6.058112,0.3368
9,2007-10-08,40.422306,33.106415,9.424173,0.3939
